In [ ]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen, Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer

url = 'https://finviz.com/quote.ashx?t=TSLA'

req = Request(url,headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
html = soup(webpage,"html.parser")

In [ ]:
pred_appended_data = tsla_data[input_vector_length:]
predictions = np.append(train_predictions, test_predictions)

pred_appended_data['Predictions'] = predictions

labels = []
for i in range(0,len(pred_appended_data)):
    row = pred_appended_data.iloc[i]
    if row['Close'] > row['Open']:
        labels.append(1)
    else:
        labels.append(0)

training_data = pred_appended_data

In [ ]:
training_data = training_data[1:] # remove the empty first row
labels = labels[1:] # remove the corresponding prediction
training_data = training_data.drop(["Date"], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(training_data, labels, test_size=0.3,random_state=109) # 70% training and 30% test

scaler = MinMaxScaler(feature_range=(0,1)).fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVC

#Create a random forest classifier
clf = SVC() # Linear Kernel


param_dict_svm = {
    'C': [0.1,1, 10, 100],
    'gamma': [1,0.1,0.01,0.001],
    'kernel': ['linear','rbf', 'poly', 'sigmoid'],
}

grid = GridSearchCV(clf,param_dict_svm,refit=True,verbose=2)
grid.fit(X_train,y_train)

y_pred = grid.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

print(accuracy_score(y_pred, y_test))
print(confusion_matrix(y_pred, y_test))